# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../Output_data/cities.csv")
cities.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kuandian,40.7286,124.7847,303.58,61,80,2.56,CN,1627101794
1,1,Etah,27.6333,78.6667,308.83,48,64,3.68,IN,1627101794
2,2,Belmonte,-15.8631,-38.8828,292.43,77,100,4.69,BR,1627101794
3,3,Kānker,20.2719,81.4931,296.62,92,100,4.27,IN,1627101795
4,4,Saint-Paul-lès-Dax,43.7272,-1.0516,290.19,97,10,1.03,FR,1627101795


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
locations = cities[["Lat", "Lng"]].astype(float)
humidity = cities["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
A max temperature lower than 80 degrees but higher than 70.

Wind speed less than 10 mph.

Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [11]:
cities

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kuandian,40.7286,124.7847,303.58,61,80,2.56,CN,1627101794
1,1,Etah,27.6333,78.6667,308.83,48,64,3.68,IN,1627101794
2,2,Belmonte,-15.8631,-38.8828,292.43,77,100,4.69,BR,1627101794
3,3,Kānker,20.2719,81.4931,296.62,92,100,4.27,IN,1627101795
4,4,Saint-Paul-lès-Dax,43.7272,-1.0516,290.19,97,10,1.03,FR,1627101795
...,...,...,...,...,...,...,...,...,...,...
563,563,Anking,30.5092,117.0506,301.29,69,34,7.99,CN,1627101974
564,564,Borovskoy,53.8000,64.1500,297.09,70,97,3.37,KZ,1627101974
565,565,Oussouye,12.4850,-16.5469,296.83,78,100,2.05,SN,1627101975
566,566,Bolshaya Glushitsa,50.2406,43.5908,295.56,44,0,2.94,RU,1627101975


In [10]:
### Weather Conditions ###
cities_narrow = cities.loc[(cities['Max Temp']<80)&(cities['Max Temp']>70)&(cities['Wind Speed']<10)&(cities['Cloudiness']==0)]
cities_narrow

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
